# Data Analysis Project: 5 - Pet Shelter Analysis

In [1]:
import pandas as pd
intakes_outcomes = pd.read_csv('./archive/aac_intakes_outcomes.csv')
intakes = pd.read_csv('./archive/aac_intakes.csv')
outcomes = pd.read_csv('./archive/aac_outcomes.csv')

In [2]:
intakes_outcomes.head(1)

,age_upon_outcome,animal_id_outcome,date_of_birth,outcome_subtype,outcome_type,sex_upon_outcome,age_upon_outcome_(days),age_upon_outcome_(years),age_upon_outcome_age_group,outcome_datetime,...,age_upon_intake_age_group,intake_datetime,intake_month,intake_year,intake_monthyear,intake_weekday,intake_hour,intake_number,time_in_shelter,time_in_shelter_days
0,10 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,3650,10.0,"(7.5, 10.0]",2017-12-07 14:07:00,...,"(7.5, 10.0]",2017-12-07 00:00:00,12,2017,2017-12,Thursday,14,1.0,0 days 14:07:00.000000000,0.588194


In [3]:
intakes.head(1)

,age_upon_intake,animal_id,animal_type,breed,color,datetime,datetime2,found_location,intake_condition,intake_type,name,sex_upon_intake
0,8 years,A706918,Dog,English Springer Spaniel,White/Liver,2015-07-05T12:59:00.000,2015-07-05T12:59:00.000,9409 Bluegrass Dr in Austin (TX),Normal,Stray,Belle,Spayed Female


In [4]:
outcomes.head()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown


___

## Meets Expectations Requirements:

### 1. Is there an area where more pets are found?
- Find the top 5 places where animals are found so the shelter can coordinate with local volunteers and animal control to monitor these areas.
#### Answer: 
    - 1: Travis :)
    - 2: 7201 Levander Loop in Austin
    - 3: Del Valle
    - 4: Pflugerville
    - 5: Manor

#### Solution:
To gather this data I utilized the `value_counts` method on the `'found_location'` series and sliced the first two entries away because they were simply `Austin (TX)`, which is where the shelter is located, and `Outside Jurisdiction` where I assume local volunteers and animal control would not be allowed to monitor.

In [5]:
# count how many times the same values repeat in this "found_location" series
locations = intakes['found_location'].value_counts()[2:7]
locations

found_location
Travis (TX)                          921
7201 Levander Loop in Austin (TX)    517
Del Valle (TX)                       411
Pflugerville (TX)                    372
Manor (TX)                           280
Name: count, dtype: int64

___

### 2-a. What is the average number of pets found in a month in the year 2015?
### 2-b. Are there months where there is a higher number of animals found?
- Knowing the number of pets the shelter might see in a month can help them gather enough resources and donations to care for the animals they receive.
#### 2-a Answer:
  - 1558

#### 2-a Solution:
First I had to separate all of the entries from the year 2015, then I gathered the amounts of entries per month of that new collection. With that I was able to find the avg of all months that year and round it to an even number.

#### 2-b Answer:
  - May
  - June
  - July
  - August
  - September
  - October

#### 2-b Solution:
For this I simply had to take my previous monthly totals for 2015 and see which months had a higher intake than the yearly average.

In [6]:
# get all intakes in 2015
twenty_fifteen = intakes_outcomes[(intakes_outcomes['intake_year'] == 2015)]
# get counts for each month, get avg and round it
month_totals = twenty_fifteen['intake_month'].value_counts()
monthly_avg = round(month_totals.mean())
monthly_avg

1558

In [7]:
high_months = month_totals[month_totals.values > monthly_avg]
high_months

intake_month
6     2188
5     2092
10    1738
8     1717
7     1634
9     1590
Name: count, dtype: int64

___

### 3. What is the ratio of incoming pets vs. adopted pets?
- This key metric helps the shelter know how they are doing.

#### Answer:
- With every 2.34 intakes, 1 pet is adopted.

#### Solution:
For this I had to find the `size` of the intakes dataframe. Then I had to create a new dataframe from the outcomes where the `outcome_type` was "Adoption" and get the `size` of that. Dividing the total intakes by this adoption dataframe size, and rounding the number a bit for easier readability, I got `2.34`.

In [8]:
intakes_total = intakes.size
adoptions_total = outcomes[outcomes['outcome_type'] == 'Adoption'].size
ratio = round(intakes_total / adoptions_total, 2)
ratio

2.34

___

### 4. What is the distribution of the types of animals in the shelter?
- Find the count of each type of animal in the shelter.

#### Answer:
- Dog: 280
- Cat: 109
- Livestock: 9
- Other: 8
- Bird: 3

#### Solution:
To solve this I first had to find out which animals are still currently in the shelter. So I `merge`d the `intakes` and `intakes_outcomes` dataframes with an inner join on their `id`s. Then I had to create a new dataframe which removes all duplicates from that merged dataframe and the `intakes` dataframe. Then I had to count how many of each value were in the `animal_type` column.

In [9]:
merged = pd.merge(intakes, intakes_outcomes, left_on='animal_id', right_on='animal_id_outcome', how='inner')
still_in_shelter = intakes[~intakes['animal_id'].isin(merged['animal_id'])]
still_in_shelter['animal_type'].value_counts()


animal_type
Dog          280
Cat          109
Livestock      9
Other          8
Bird           3
Name: count, dtype: int64

___

### 5. What are the adoption rates for specific breeds?
- Find the top 5 dog breeds in the shelter (based on count) and then find the adoption percentage of each breed.

#### Answer:

#### Solution:
First I had to find the top 5 dog breeds still in the shelter so I used my previous `still_in_shelter` dataframe.

In [23]:
top_five_dogs = still_in_shelter[still_in_shelter['animal_type'] == 'Dog'].breed.value_counts().head()
top_five_dogs

breed
Pit Bull Mix                     86
Labrador Retriever Mix           44
Chihuahua Shorthair Mix          21
American Pit Bull Terrier Mix    13
German Shepherd Mix              10
Name: count, dtype: int64

### 6. What are the adoption rates for different colorings?
- Find the top 5 colorings in the shelter (based on count) and then find the adoption percentage of each color.

### 7. About how many animals are spayed/neutered each month?
- This will help the shelter allocate resources and staff. Assume that all intact males and females will be spayed/neutered.

## Exceeds Expectations Requirements:

### 1-a. How many animals in the shelter are repeats?

### 1-b. Which animal was returned to the shelter the most?

### 2. What are the adoption rates for the following age groups?
- baby: 4 months and less
- young: 5 months - 2 years
- adult: 3 years - 10 years
- senior: 11+

### 3. If spay/neuter for a dog costs $100 and a spay/neuter for a cat costs $50, how much did the shelter spend in 2015 on these procedures?